# Coding Assignment 2

Authors:
* Ryan Fogle
    - rsfogle2@illinois.edu
    - UIN: 652628818
* Sean Enright
    - seanre2@illinois.edu
    - UNIN: 661791377

## Part 1: Implement Lasso

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

### Load Data

In [ ]:
myData = pd.read_csv("Coding2_Data.csv")
var_names = myData.columns
y = myData[['Y']].to_numpy()
X = myData.drop(['Y'], axis = 1).to_numpy()

In [ ]:
X.shape, len(y)

### CD for Lasso

In [ ]:
import numpy.typing as npt

def one_var_lasso(r: npt.NDArray, x: npt.NDArray, lam):
    
    #################
    # Your CODE
    #################
    
    # x == z
    # v == r

    z2 = (x.T @ x).sum()
    a = r.T @ x / z2
    n = 2 * x.shape[0] * lam / z2

    if a > n/2:
        return a - n/2
    elif np.abs(a) <= n/2:
        return 0
    elif a < -n/2:
        return a + n/2

In [ ]:
from sklearn.preprocessing import StandardScaler

def MyLasso(X, y, lam_seq, maxit = 100):
    
    # Input
    # X: n-by-p design matrix without the intercept 
    # y: n-by-1 response vector 
    # lam.seq: sequence of lambda values (arranged from large to small)
    # maxit: number of updates for each lambda 
    
    # Output
    # B: a (p+1)-by-len(lam.seq) coefficient matrix 
    #    with the first row being the intercept sequence 

  
    n, p = X.shape
    nlam = len(lam_seq)
    B = np.zeros((p+1, nlam))
    
    ##############################
    # YOUR CODE: 
    # (1) newX = Standardizad X; 
    # (2) Record the centers and scales used in (1) 
    ##############################
    
    # Centers
    y_mean = y.mean()
    X_mean = X.mean(axis=0)

    # Scale
    X_se = X.std(axis=0)

    # Centering and scaling of X
    newX = (X - X_mean) / X_se

    # Initilize coef vector b and residual vector r
    b = np.zeros(p)
    r = y
    
    # Triple nested loop
    for m in range(nlam):
        for step in range(maxit):
            for j in range(p):
                X_j = newX[:, j].reshape(-1,1)
                r = r + X_j * b[j]
                b[j] = one_var_lasso(r, X_j, lam_seq[m])
                r = r - X_j * b[j]
        B[1:, m] = b 
    
    ##############################
    # YOUR CODE:
    # Scale back the coefficients;
    # Update the intercepts stored in B[, 1]
    ##############################

    # Scale back
    B[1:, :] = B[1:, :] / X_se[:, np.newaxis]
    
    # Add in new intercepts
    B[0, :] = y_mean - (X_mean[np.newaxis, :] @ B[1:, :]).squeeze()
    
    return(B)

In [ ]:
log_lam_seq = np.linspace(-1, -8, num = 80)
lam_seq = np.exp(log_lam_seq)
myout = MyLasso(X, y, lam_seq, maxit = 100)

In [ ]:
p, _ = myout.shape
plt.figure(figsize = (12,8))

for i in range(p-1):
    plt.plot(log_lam_seq, myout[i+1, :], label = var_names[i])

plt.xlabel('Log Lambda')
plt.ylabel('Coefficients')
plt.title('Lasso Paths - Numpy implementation')
plt.legend()
plt.axis('tight')

### Check the Accuracy
The output of our algorithm is compared against the output from glmnet. The maximum difference between the two coefficient matrices should be lass than 0.005.

In [ ]:
lasso_coef = pd.read_csv("Coding2_lasso_coefs.csv").to_numpy()
lasso_coef.shape

In [ ]:
abs(myout - lasso_coef).max()

## Part II: Simulation Study

### Code for Six Procedures

The following code is shared by both Case I and Case II below.

Six prediction procedures are defined:
* Linear regression with all features
* Ridge regression using `lambda.min`
* Lasso with `lambda.min`
* Lasso with `lambda.1se`
* Lasso refit from the model with `lambda.1se`
* Principal components regression



In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge, RidgeCV, Lasso, LassoCV

from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

In [ ]:
class PCR(object):

    def __init__(self, num_folds=10):
        self.folds = num_folds

    def fit(self, X, Y):
        n, p = X.shape
        indices = np.arange(n)
        np.random.shuffle(indices)
        index_sets = np.array_split(indices, self.folds)
        ncomp = min(p, n - 1 - max([len(i) for i in index_sets]))
        cv_err = np.zeros(ncomp)

        for ifold in range(self.folds):
            train_inds =  np.delete(index_sets, obj=ifold, axis=0).ravel()
            test_inds = index_sets[ifold]

            X_train = X[train_inds, :]
            pipeline = Pipeline([('scaling', StandardScaler()), ('pca', PCA())])
            pipeline.fit(X_train)
            X_train = pipeline.transform(X_train)
            coefs = Y[train_inds].T @ X_train / np.sum(X_train**2, axis=0)
            b0 = np.mean(Y[train_inds])

            X_test = pipeline.transform(X[test_inds, :])

            for k in np.arange(ncomp):
                preds = X_test[:, :k] @ coefs.T[:k] + b0
                cv_err[k] += cv_err[k] + np.sum((Y[test_inds]-preds)**2)

        min_ind = np.argmin(cv_err)
        self.ncomp = min_ind+1
        pipeline = Pipeline([('scaling', StandardScaler()), ('pca', PCA(n_components=self.ncomp))])
        self.transform = pipeline.fit(X)
        self.model = LinearRegression().fit(self.transform.transform(X), Y)

    def predict(self, X):
        X_ = self.transform.transform(X)
        return self.model.predict(X_)

In [ ]:
class LassoBase:
    """Predict with Lasso Regression variants
        Note: X_train and X_test must be centered and scaled"""
    def __init__(self, X_train, y_train):
        self.X_train = X_train
        self.y_train = y_train

        # Lasso
        lasso_alphas = np.logspace(-10, 1, 100)
        lassocv = LassoCV(alphas = lasso_alphas, cv = 10)
        lassocv.fit(X_train, y_train)
        cv_alphas = lassocv.alphas_
        mean_mse = np.mean(lassocv.mse_path_, axis=1)
        min_idx = np.argmin(mean_mse)

        # Minimum
        self.alpha_min = cv_alphas[min_idx]

        # 1se
        std_mse = np.std(lassocv.mse_path_, axis=1) / np.sqrt(10) 
        threshold = mean_mse[min_idx] + std_mse[min_idx]
        self.alpha_1se = max(cv_alphas[np.where(mean_mse <= threshold)])

        # Refit
        self.nonzero_indices = None
    
    def min(self, X_test):
        """Regression with lambda.min"""
        lasso_model_min = Lasso(alpha = self.alpha_min, max_iter=10000)
        lasso_model_min.fit(self.X_train, self.y_train)
        return lasso_model_min.predict(X_test)
    
    def one_se(self, X_test):
        """Regression with lambda.1se"""
        lasso_model_1se = Lasso(alpha = self.alpha_1se, max_iter=10000)
        lasso_model_1se.fit(self.X_train, self.y_train)
        self.nonzero_indices = np.where(lasso_model_1se.coef_ != 0)[0]
        return lasso_model_1se.predict(X_test)
    
    def refit(self, X_test):
        """Refit regression from lambda.1se"""
        lm_refit = LinearRegression()
        lm_refit.fit(self.X_train.iloc[:, self.nonzero_indices],
                     self.y_train)
        return lm_refit.predict(X_test.iloc[:, self.nonzero_indices])


def full_model(X_train, y_train, X_test):
    """Predict with Full Linear Model"""
    full = LinearRegression()
    full.fit(X_train, y_train)
    return full.predict(X_test)

def ridge_regression(X_train, y_train, X_test):
    """Predict with Ridge Regression
        Note: X_train and X_test must be centered and scaled"""
    # Ridge regression
    ridge_alphas = np.logspace(-10, 1, 100)
    ridgecv = RidgeCV(alphas = ridge_alphas, cv = 10,
                      scoring = 'neg_mean_squared_error')
    ridgecv.fit(X_train, y_train)
    ridge_model = Ridge(alpha = ridgecv.alpha_)
    ridge_model.fit(X_train, y_train)
    return ridge_model.predict(X_test)

def principal_component_regression(X_train, y_train, X_test):
    # perform PCR and train linear model.
    pcr = PCR()
    pcr.fit(X_train.to_numpy(), y_train.to_numpy())
    return pcr.predict(X_test.to_numpy())

In [ ]:
def gen_sim_data(X, Y, pct_test=0.25):
    n = len(Y)
    indices = np.arange(0, n)
    np.random.shuffle(indices)
    test_ind = indices[:int(np.floor(pct_test * n))]
    train_ind = indices[len(test_ind):]

    # Splitting the data into training and testing sets
    X_train = X.iloc[train_ind]
    y_train = Y[train_ind]
    X_test = X.iloc[test_ind]
    y_test = Y[test_ind]

    # We need to scale data for Ridge and Lasso because they cannot normalize like R. 
    # Scaler
    scaler = StandardScaler()
    X_train_scale = scaler.fit_transform(X_train)
    X_test_scale = scaler.transform(X_test)
    
    return X_train, y_train, X_test, y_test, X_train_scale, X_test_scale

### Case I

This simulation uses the data in `Coding2_Data2.csv`, which has 91 columns (1 response and 90 predictors), and 506 observations. The first 14 columns are the same data used in Part I.

In [ ]:
url = "https://raw.githubusercontent.com/liangfgithub/liangfgithub.github.io/master/Data/Coding2_Data2.csv"
myData = pd.read_csv(url)
# myData.head()
Y = myData['Y']
X = myData.drop(['Y'], axis = 1)

In [ ]:
X.shape, len(Y)

#### Simulation

In this simulation, each of the six models is used to for prediction with the `Coding2_Data2.csv` data, and their error is compared via mean squared prediction error (MSPE). The following procedure is carried out for 50 iterations:
1) Partition data into 75% training and 25% test sets
2) Fit training data with all six models, including:
    * Linear regression with all features
    * Ridge regression using `lambda.min`
    * Lasso with `lambda.min`
    * Lasso with `lambda.1se`
    * Lasso refit from the model with `lambda.1se`
    * Principal components regression
3) Predict response with test data
4) Evaluate MSPE

In [ ]:
from tqdm import tqdm
# from sklearn.utils.testing import ignore_warnings
from sklearn.exceptions import ConvergenceWarning
import warnings

# Suppress convergence errors
warnings.filterwarnings(action='ignore', category=ConvergenceWarning)

In [ ]:
n_models = 6
n_sims = 50
data = []

for i in tqdm(range(n_sims), total=n_sims):
    X_train, y_train, X_test, y_test, X_train_scale, X_test_scale = gen_sim_data(X, Y,
                                                                                 pct_test=0.25)
    lasso = LassoBase(X_train, y_train)
    predictions = (
        mean_squared_error(y_test, full_model(X_train, y_train, X_test)),
        mean_squared_error(y_test, ridge_regression(X_train_scale, y_train, X_test_scale)),
        mean_squared_error(y_test, lasso.min(X_test)),
        mean_squared_error(y_test, lasso.one_se(X_test)),
        mean_squared_error(y_test, lasso.refit(X_test)),
        mean_squared_error(y_test, principal_component_regression(X_train, y_train, X_test))
    )
    data.append(predictions)
    
df = pd.DataFrame(data, columns=['Full', 'Ridge.min', 'Lasso.min', 'Lasso.1se', 'L.Refit', 'PCR'])
df.head()

#### Graphical Summary

The MSPE for each model is presented in the boxplot and strip charts below.

In [ ]:
import seaborn as sns

# Strip chart
plt.title('MSPE by Model')
sns.stripplot(df)
plt.show()

In [ ]:
# Box plot
plt.title('MSPE by Model')
sns.boxplot(df)
plt.show()